In [1]:
import pandas as pd
import seaborn as sns
import numpy as np

import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

import re

##### Read features as dataframes

In [2]:
year_range = [f'{i}_{i+2}' for i in range(1990,2020,3)]

In [3]:
region_en = 'xibei'

In [4]:
# import the points with image values 
feature_df = [pd.read_csv(f'./Data/Training_sample_{region_en}_{year}.csv') for year in year_range]

# merge all pt_with_img and remove unnecessary columns
feature_df = [fe.drop(['system:index','.geo'],axis=1) for fe in feature_df]

##### train_test_split and apply randomforest_classifier

In [ ]:
Sample_acc = {}

for year,fe in zip(year_range,feature_df):
    
    # _______________________________Different band combination_______________________________
    
    col_name = ''.join(fe.columns)
    
    # find the spectrum band names
    Spectrum_reg = re.compile(r'(B6_VCID_\d|B\d{1,2}|Landsat_B\d{1,2}|Sentinel_B\d{1,2}A?)')
    Spectrum_reg_bands = Spectrum_reg.findall(col_name)
    
    # find the Mean-nomalized names
    Mean_nomalize_bands = ['NDVI','NDBI','EVI']
    
    # find the Fourire names
    Fourier_reg   = re.compile(r'((EVI|NDBI|NDVI)_(constant|\w{3}_\d))')
    Fourier_bands = [i[0] for i in Fourier_reg.findall(col_name)]
    
    # get the terrain bands
    Terrain_bands = ['DEM','SLOPE']
    
    # get the meterology bands
    Meterology_bands = ['lrad', 'prec', 'pres', 'shum', 'srad', 'temp', 'wind']

    
    #_________________________________Assess the accuracy ______________________________________
    
    bands_in = zip(['Spectrum',
                    'Spectrum_Normalize',
                    'Spectrum_Normalize_Fourier',
                    'Spectrum_Normalize_Fourier_Terrain',
                    'Spectrum_Normalize_Fourier_Terrain_Meterology'],
                   
                   [Spectrum_reg_bands,
                    Spectrum_reg_bands + Mean_nomalize_bands,
                    Spectrum_reg_bands + Mean_nomalize_bands + Fourier_bands,
                    Spectrum_reg_bands + Mean_nomalize_bands + Fourier_bands + Terrain_bands,
                    Spectrum_reg_bands + Mean_nomalize_bands + Fourier_bands + Terrain_bands + Meterology_bands])
    
    # each_year_band_combination[year] =  list(bands_in)
    
    # ________________________Grid search to compare the accuracy under different input bands__________
    
    for band_name, bands_select in bands_in:
        
        # get the X and y
        X_train = fe[bands_select]
        y_train = fe['Built']
        
        # import testing sample
        test_sample = pd.read_csv(f'./Data/Grid_select_{region_en}_{year}.csv')
        test_sample = test_sample.drop(['system:index','.geo'],axis=1)
        # get the X,y value
        X_test = test_sample.drop('Built',1)[bands_select]
        y_test = test_sample['Built']     
        
        
        # _______________________Grid search to compare the accuracy under different trees__________
        
        for tree in range(1,121):

            # perform the RF classification
            RF = RandomForestClassifier(n_estimators=tree,n_jobs=-1)
            RF.fit(X_train,y_train)
            y_pred = RF.predict(X_test)

            # get the accuracy, here get two kinds of accuracy
            acc_test = accuracy_score(y_test,y_pred)

            # add results to the acc_dict
            Sample_acc[(year,band_name,tree)] = [acc_test]

            # print out the process
            print(f'Accuracy calculation of {region_en}_{year}_{band_name}_{tree} finished!')
        

Accuracy calculation of xibei_1990_1992_Spectrum_1 finished!
Accuracy calculation of xibei_1990_1992_Spectrum_2 finished!
Accuracy calculation of xibei_1990_1992_Spectrum_3 finished!
Accuracy calculation of xibei_1990_1992_Spectrum_4 finished!
Accuracy calculation of xibei_1990_1992_Spectrum_5 finished!
Accuracy calculation of xibei_1990_1992_Spectrum_6 finished!
Accuracy calculation of xibei_1990_1992_Spectrum_7 finished!
Accuracy calculation of xibei_1990_1992_Spectrum_8 finished!
Accuracy calculation of xibei_1990_1992_Spectrum_9 finished!
Accuracy calculation of xibei_1990_1992_Spectrum_10 finished!
Accuracy calculation of xibei_1990_1992_Spectrum_11 finished!
Accuracy calculation of xibei_1990_1992_Spectrum_12 finished!
Accuracy calculation of xibei_1990_1992_Spectrum_13 finished!
Accuracy calculation of xibei_1990_1992_Spectrum_14 finished!
Accuracy calculation of xibei_1990_1992_Spectrum_15 finished!
Accuracy calculation of xibei_1990_1992_Spectrum_16 finished!
Accuracy calculat

Accuracy calculation of xibei_1990_1992_Spectrum_Normalize_12 finished!
Accuracy calculation of xibei_1990_1992_Spectrum_Normalize_13 finished!
Accuracy calculation of xibei_1990_1992_Spectrum_Normalize_14 finished!
Accuracy calculation of xibei_1990_1992_Spectrum_Normalize_15 finished!
Accuracy calculation of xibei_1990_1992_Spectrum_Normalize_16 finished!
Accuracy calculation of xibei_1990_1992_Spectrum_Normalize_17 finished!
Accuracy calculation of xibei_1990_1992_Spectrum_Normalize_18 finished!
Accuracy calculation of xibei_1990_1992_Spectrum_Normalize_19 finished!
Accuracy calculation of xibei_1990_1992_Spectrum_Normalize_20 finished!
Accuracy calculation of xibei_1990_1992_Spectrum_Normalize_21 finished!
Accuracy calculation of xibei_1990_1992_Spectrum_Normalize_22 finished!
Accuracy calculation of xibei_1990_1992_Spectrum_Normalize_23 finished!
Accuracy calculation of xibei_1990_1992_Spectrum_Normalize_24 finished!
Accuracy calculation of xibei_1990_1992_Spectrum_Normalize_25 fi

Accuracy calculation of xibei_1990_1992_Spectrum_Normalize_Fourier_6 finished!
Accuracy calculation of xibei_1990_1992_Spectrum_Normalize_Fourier_7 finished!
Accuracy calculation of xibei_1990_1992_Spectrum_Normalize_Fourier_8 finished!
Accuracy calculation of xibei_1990_1992_Spectrum_Normalize_Fourier_9 finished!
Accuracy calculation of xibei_1990_1992_Spectrum_Normalize_Fourier_10 finished!
Accuracy calculation of xibei_1990_1992_Spectrum_Normalize_Fourier_11 finished!
Accuracy calculation of xibei_1990_1992_Spectrum_Normalize_Fourier_12 finished!
Accuracy calculation of xibei_1990_1992_Spectrum_Normalize_Fourier_13 finished!
Accuracy calculation of xibei_1990_1992_Spectrum_Normalize_Fourier_14 finished!
Accuracy calculation of xibei_1990_1992_Spectrum_Normalize_Fourier_15 finished!
Accuracy calculation of xibei_1990_1992_Spectrum_Normalize_Fourier_16 finished!
Accuracy calculation of xibei_1990_1992_Spectrum_Normalize_Fourier_17 finished!
Accuracy calculation of xibei_1990_1992_Spec

Accuracy calculation of xibei_1990_1992_Spectrum_Normalize_Fourier_109 finished!
Accuracy calculation of xibei_1990_1992_Spectrum_Normalize_Fourier_110 finished!
Accuracy calculation of xibei_1990_1992_Spectrum_Normalize_Fourier_111 finished!
Accuracy calculation of xibei_1990_1992_Spectrum_Normalize_Fourier_112 finished!
Accuracy calculation of xibei_1990_1992_Spectrum_Normalize_Fourier_113 finished!
Accuracy calculation of xibei_1990_1992_Spectrum_Normalize_Fourier_114 finished!
Accuracy calculation of xibei_1990_1992_Spectrum_Normalize_Fourier_115 finished!
Accuracy calculation of xibei_1990_1992_Spectrum_Normalize_Fourier_116 finished!
Accuracy calculation of xibei_1990_1992_Spectrum_Normalize_Fourier_117 finished!
Accuracy calculation of xibei_1990_1992_Spectrum_Normalize_Fourier_118 finished!
Accuracy calculation of xibei_1990_1992_Spectrum_Normalize_Fourier_119 finished!
Accuracy calculation of xibei_1990_1992_Spectrum_Normalize_Fourier_120 finished!
Accuracy calculation of xibe

Accuracy calculation of xibei_1990_1992_Spectrum_Normalize_Fourier_Terrain_84 finished!
Accuracy calculation of xibei_1990_1992_Spectrum_Normalize_Fourier_Terrain_85 finished!
Accuracy calculation of xibei_1990_1992_Spectrum_Normalize_Fourier_Terrain_86 finished!
Accuracy calculation of xibei_1990_1992_Spectrum_Normalize_Fourier_Terrain_87 finished!
Accuracy calculation of xibei_1990_1992_Spectrum_Normalize_Fourier_Terrain_88 finished!
Accuracy calculation of xibei_1990_1992_Spectrum_Normalize_Fourier_Terrain_89 finished!
Accuracy calculation of xibei_1990_1992_Spectrum_Normalize_Fourier_Terrain_90 finished!
Accuracy calculation of xibei_1990_1992_Spectrum_Normalize_Fourier_Terrain_91 finished!
Accuracy calculation of xibei_1990_1992_Spectrum_Normalize_Fourier_Terrain_92 finished!
Accuracy calculation of xibei_1990_1992_Spectrum_Normalize_Fourier_Terrain_93 finished!
Accuracy calculation of xibei_1990_1992_Spectrum_Normalize_Fourier_Terrain_94 finished!
Accuracy calculation of xibei_19

Accuracy calculation of xibei_1990_1992_Spectrum_Normalize_Fourier_Terrain_Meterology_51 finished!
Accuracy calculation of xibei_1990_1992_Spectrum_Normalize_Fourier_Terrain_Meterology_52 finished!
Accuracy calculation of xibei_1990_1992_Spectrum_Normalize_Fourier_Terrain_Meterology_53 finished!
Accuracy calculation of xibei_1990_1992_Spectrum_Normalize_Fourier_Terrain_Meterology_54 finished!
Accuracy calculation of xibei_1990_1992_Spectrum_Normalize_Fourier_Terrain_Meterology_55 finished!
Accuracy calculation of xibei_1990_1992_Spectrum_Normalize_Fourier_Terrain_Meterology_56 finished!
Accuracy calculation of xibei_1990_1992_Spectrum_Normalize_Fourier_Terrain_Meterology_57 finished!
Accuracy calculation of xibei_1990_1992_Spectrum_Normalize_Fourier_Terrain_Meterology_58 finished!
Accuracy calculation of xibei_1990_1992_Spectrum_Normalize_Fourier_Terrain_Meterology_59 finished!
Accuracy calculation of xibei_1990_1992_Spectrum_Normalize_Fourier_Terrain_Meterology_60 finished!
Accuracy c

Accuracy calculation of xibei_1993_1995_Spectrum_22 finished!
Accuracy calculation of xibei_1993_1995_Spectrum_23 finished!
Accuracy calculation of xibei_1993_1995_Spectrum_24 finished!
Accuracy calculation of xibei_1993_1995_Spectrum_25 finished!
Accuracy calculation of xibei_1993_1995_Spectrum_26 finished!
Accuracy calculation of xibei_1993_1995_Spectrum_27 finished!
Accuracy calculation of xibei_1993_1995_Spectrum_28 finished!
Accuracy calculation of xibei_1993_1995_Spectrum_29 finished!
Accuracy calculation of xibei_1993_1995_Spectrum_30 finished!
Accuracy calculation of xibei_1993_1995_Spectrum_31 finished!
Accuracy calculation of xibei_1993_1995_Spectrum_32 finished!
Accuracy calculation of xibei_1993_1995_Spectrum_33 finished!
Accuracy calculation of xibei_1993_1995_Spectrum_34 finished!
Accuracy calculation of xibei_1993_1995_Spectrum_35 finished!
Accuracy calculation of xibei_1993_1995_Spectrum_36 finished!
Accuracy calculation of xibei_1993_1995_Spectrum_37 finished!
Accuracy

Accuracy calculation of xibei_1993_1995_Spectrum_Normalize_30 finished!
Accuracy calculation of xibei_1993_1995_Spectrum_Normalize_31 finished!
Accuracy calculation of xibei_1993_1995_Spectrum_Normalize_32 finished!
Accuracy calculation of xibei_1993_1995_Spectrum_Normalize_33 finished!
Accuracy calculation of xibei_1993_1995_Spectrum_Normalize_34 finished!
Accuracy calculation of xibei_1993_1995_Spectrum_Normalize_35 finished!
Accuracy calculation of xibei_1993_1995_Spectrum_Normalize_36 finished!
Accuracy calculation of xibei_1993_1995_Spectrum_Normalize_37 finished!
Accuracy calculation of xibei_1993_1995_Spectrum_Normalize_38 finished!
Accuracy calculation of xibei_1993_1995_Spectrum_Normalize_39 finished!
Accuracy calculation of xibei_1993_1995_Spectrum_Normalize_40 finished!
Accuracy calculation of xibei_1993_1995_Spectrum_Normalize_41 finished!
Accuracy calculation of xibei_1993_1995_Spectrum_Normalize_42 finished!
Accuracy calculation of xibei_1993_1995_Spectrum_Normalize_43 fi

Accuracy calculation of xibei_1993_1995_Spectrum_Normalize_Fourier_22 finished!
Accuracy calculation of xibei_1993_1995_Spectrum_Normalize_Fourier_23 finished!
Accuracy calculation of xibei_1993_1995_Spectrum_Normalize_Fourier_24 finished!
Accuracy calculation of xibei_1993_1995_Spectrum_Normalize_Fourier_25 finished!
Accuracy calculation of xibei_1993_1995_Spectrum_Normalize_Fourier_26 finished!
Accuracy calculation of xibei_1993_1995_Spectrum_Normalize_Fourier_27 finished!
Accuracy calculation of xibei_1993_1995_Spectrum_Normalize_Fourier_28 finished!
Accuracy calculation of xibei_1993_1995_Spectrum_Normalize_Fourier_29 finished!


##### Save the grid_acc  to disk

In [ ]:
grid_acc = pd.DataFrame(Sample_acc).T
grid_acc.reset_index(inplace=True)
grid_acc.columns = ['Year','In_Bands','Tree','Accuracy']
grid_acc.to_csv(f'./Result/Grid_acc_tree_{region_en}.csv',index = False)

# load the grid_acc
grid_acc = pd.read_csv(f'./Result/Grid_acc_tree_{region_en}.csv')
grid_acc['Year'] = grid_acc['Year'].astype('category')

In [ ]:
grid_acc = pd.read_csv(f'./Result/Grid_acc_tree_{region_en}.csv')
grid_acc['Year'] = grid_acc['Year'].astype('category')

##### Observe that the tree number of 100 is enough for classification

In [ ]:
plt.figure(figsize=(12,8))
sns.lineplot(data = grid_acc[grid_acc['In_Bands']=='Spectrum_Normalize_Fourier_Terrain_Meterology'],
             x='Tree',
             y='Accuracy')

##### Observe that the Fourier bands significant increases the accuracy

In [ ]:
# get the accuracy with the tree number of 100
Acc_tree_100 = grid_acc[grid_acc['Tree'] == 100]

In [ ]:
plt.figure(figsize=(12,8))
sns.lineplot(data = Acc_tree_100,
             x='Year',
             y='Accuracy',
             hue = 'In_Bands')

##### Compare all region's accuracy

In [ ]:
All_accuracy = []

for region in ['dongbei','huadong','zhongnan','xinan','xibei','NCP']:
    
    df = pd.read_csv(f'./Result/Grid_acc_tree_{region}.csv')
    df['Region'] = region
    All_accuracy.append(df)

In [ ]:
All_accurac_df_tree_100 = All_accurac_df[All_accurac_df['Tree'] == 100]

In [ ]:
sns.relplot(data=All_accurac_df_tree_100,x='Year',y='Accuracy',hue='In_Bands',col='Region',kind="line",col_wrap=3)